In [ ]:
"""
Escolha uma base de dados qualquer do site kaggle.com, tarefa de classificação
o arquivo é o Student_performance_data_.csv

Escolha um classificador do tipo ensamble

Rode o classificador na base de dados e estime a performace de generalização

Use método de hold-out para avaliação do modelo (treino/teste)

Use método 10-fold para seleção do modelo (treino validação)

Selecione o melhor modelo (hyperparâmetros) usando gridsearch

Utilizando o modelo selecionado, avalie se existe overfitting

Calcule intervalos de confiança via aproximação normal
"""

import pandas as pd
from sklearn.model_selection import train_test_split, KFold, GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
from scipy import stats

# Load data
data = pd.read_csv('Student_performance_data_.csv')

# preprocessamento
data = data.dropna()
data = pd.get_dummies(data)

# Separar os dados em conjuntos de treino e teste (método hold-out)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Definir o modelo de Random Forest
rf = RandomForestRegressor(random_state=42)

# Selecionar o melhor modelo (hiperparâmetros) usando GridSearchCV com validação cruzada 10-fold
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10]
}

grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=10, scoring='r2', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Melhor modelo encontrado pelo GridSearchCV
best_rf = grid_search.best_estimator_
print("Melhores hiperparâmetros:", grid_search.best_params_)

# Avaliar o modelo no conjunto de teste para verificar overfitting
y_train_pred = best_rf.predict(X_train)
y_test_pred = best_rf.predict(X_test)

mse_train = mean_squared_error(y_train, y_train_pred)
mse_test = mean_squared_error(y_test, y_test_pred)

r2_train = r2_score(y_train, y_train_pred)
r2_test = r2_score(y_test, y_test_pred)

print(f'Treinamento - MSE: {mse_train}, R²: {r2_train}')
print(f'Teste - MSE: {mse_test}, R²: {r2_test}')

# Verificar se há overfitting comparando a performance no treino e no teste
if r2_train > r2_test:
    print("Possível overfitting detectado.")

# Calcular intervalos de confiança via aproximação normal
scores = cross_val_score(best_rf, X, y, cv=10, scoring='r2')
mean_score = np.mean(scores)
std_dev = np.std(scores)

confidence_interval = stats.norm.interval(0.95, loc=mean_score, scale=std_dev/np.sqrt(len(scores)))

print(f'R² Cross-Validated Score: {mean_score}')
print(f'95% Confidence Interval for R²: {confidence_interval}')


Melhores hiperparâmetros: {'max_depth': None, 'min_samples_split': 10, 'n_estimators': 100}
Treinamento - MSE: 0.06633323463562492, R²: 0.9563831480834202
Teste - MSE: 0.23746970554563201, R²: 0.8441844304727919
Possível overfitting detectado.
R² Cross-Validated Score: 0.8077789908023847
95% Confidence Interval for R²: (0.45193691901810096, 1.1636210625866683)
